In [ ]:
# ============================================================
# Logistic Regression (PyTorch) + Multiple Optimizers
# Comparing SGD, SGD+Momentum, Adam, RMSprop, LBFGS
# with 60/20/20 train/val/test split
# ============================================================

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# ------------------------------------------------------------
# 1. (Example) Create or load your dataset
# ------------------------------------------------------------
n = 10

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# 60 / 20 / 20 split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, stratify=y, random_state=42
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")

# ------------------------------------------------------------
# 2. Preprocessing
# ------------------------------------------------------------
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# ------------------------------------------------------------
# 3. Logistic Regression Model
# ------------------------------------------------------------
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        return torch.sigmoid(self.linear(x))


# ------------------------------------------------------------
# 4. Training & Evaluation Function
# ------------------------------------------------------------
def train_model(optimizer_name, model, X_train, y_train, X_val, y_val,
                lr=1e-3, weight_decay=0.0, momentum=0.9, epochs=50, batch_size=64):
    criterion = nn.BCELoss()

    # Initialize optimizer
    if optimizer_name == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "SGD+Momentum":
        optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)
    elif optimizer_name == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=lr, weight_decay=weight_decay)
    elif optimizer_name == "LBFGS":
        optimizer = optim.LBFGS(model.parameters(), lr=lr, max_iter=20)
    else:
        raise ValueError("Unknown optimizer")

    n = len(X_train)
    for epoch in range(epochs):
        model.train()
        idx = torch.randperm(n)
        X_train, y_train = X_train[idx], y_train[idx]

        if optimizer_name == "LBFGS":
            # LBFGS requires closure
            def closure():
                optimizer.zero_grad()
                preds = model(X_train)
                loss = criterion(preds, y_train)
                loss.backward()
                return loss
            optimizer.step(closure)
        else:
            for i in range(0, n, batch_size):
                xb = X_train[i:i+batch_size]
                yb = y_train[i:i+batch_size]
                optimizer.zero_grad()
                preds = model(xb)
                loss = criterion(preds, yb)
                loss.backward()
                optimizer.step()

    # Validation accuracy
    model.eval()
    with torch.no_grad():
        preds_val = model(X_val)
    preds_bin = (preds_val > 0.5).float()
    acc = accuracy_score(y_val, preds_bin)
    return acc


# ------------------------------------------------------------
# 5. Compare Optimizers
# ------------------------------------------------------------
optimizers = ["SGD", "SGD+Momentum", "Adam", "RMSprop", "LBFGS"]
results = {}

for opt in optimizers:
    model = LogisticRegressionModel(X_train.shape[1])
    acc = train_model(opt, model, X_train, y_train, X_val, y_val,
                      lr=1e-3 if opt != "LBFGS" else 1.0,
                      weight_decay=1e-4)
    results[opt] = acc
    print(f"{opt:12s} → Validation Accuracy: {acc:.4f}")

# ------------------------------------------------------------
# 6. Select Best Optimizer & Test
# ------------------------------------------------------------
best_opt = max(results, key=results.get)
print("\nBest Optimizer:", best_opt)

# Retrain best model on Train + Val, Evaluate on Test
model_best = LogisticRegressionModel(X_train.shape[1])
X_combined = torch.cat([X_train, X_val], dim=0)
y_combined = torch.cat([y_train, y_val], dim=0)

_ = train_model(best_opt, model_best, X_combined, y_combined, X_test, y_test,
                lr=1e-3 if best_opt != "LBFGS" else 1.0, weight_decay=1e-4)

model_best.eval()
with torch.no_grad():
    preds_test = model_best(X_test)
preds_bin = (preds_test > 0.5).float()
test_acc = accuracy_score(y_test, preds_bin)

print(f"\nFinal Test Accuracy (using {best_opt}): {test_acc:.4f}")

# ------------------------------------------------------------
# 7. Summary
# ------------------------------------------------------------
print("\n=== Optimizer Comparison Summary ===")
for opt, acc in results.items():
    print(f"{opt:12s} → Val Accuracy: {acc:.4f}")
print(f"\nBest Optimizer: {best_opt} | Test Accuracy: {test_acc:.4f}")
